# ES → BBF Salesforce Location Migration

This notebook migrates Address__c records from ES Salesforce to Location__c in BBF Salesforce.

## Prerequisites
- **Account migration must be completed first** (Accounts have BBF_New_Id__c populated)
- ES Address__c must have BBF_New_Id__c field created

## What This Notebook Does
1. Queries ES Address__c records that haven't been migrated yet
2. Transforms field names/values from ES to BBF format
3. Inserts Location__c records into BBF
4. Updates ES Address__c with the new BBF Location ID (BBF_New_Id__c)
5. Outputs Excel file with results, mapping, and any failures

## BBF Location__c Required Fields
- `Name_Is_Set_Manually__c` (boolean) - **REQUIRED** - Set to False

## Field Mapping
| BBF Location__c | ES Address__c | Notes |
|-----------------|---------------|-------|
| Name | Name | Address Name |
| Name_Is_Set_Manually__c | (default False) | REQUIRED |
| City__c | City__c | Exact match |
| State__c | State__c | Exact match |
| County__c | County__c | Exact match |
| PostalCode__c | Zip__c | Different API name |
| Street__c | Address__c | Different API name |
| Full_Address__c | Complete_Address__c | Different API name |
| Loc__c | Geocode_Lat_Long__c | Geolocation field |
| CLLICode__c | CLLI__c | Different API name |
| ES_Legacy_ID__c | Id | For tracking |

In [14]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Set-up successful


In [15]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "vlettau@everstream.net.uat"
ES_PASSWORD = "MNlkpo0987)(*&"
ES_TOKEN = "nSBoNS97wYLCRW2JP2JARR12"
ES_DOMAIN = "test"  # or 'login' for production

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = True  # ⚠️ Set to False to migrate ALL Locations
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# Filter Option: Only migrate addresses used by active Orders
FILTER_ACTIVE_ORDERS_ONLY = (
    False  # Set to True to only migrate addresses referenced by active Orders
)

# 👤 Location Owner - Set all migrated Locations to this user
OWNER_ID = "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN migration

# 🏢 Default Business Unit - Picklist field in BBF Location__c
DEFAULT_BUS_UNIT = "EVS"  # EverStream business unit in BBF

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_location_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   FILTER_ACTIVE_ORDERS_ONLY: {FILTER_ACTIVE_ORDERS_ONLY}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Default Business Unit: {DEFAULT_BUS_UNIT}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   TEST_MODE: True
   FILTER_ACTIVE_ORDERS_ONLY: False
   Owner ID: 005Ea00000ZOGFZIA5
   Default Business Unit: EVS
   Output: es_bbf_location_migration_20251212_145550.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [16]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

📌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

📌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [17]:
# === QUERY ES ADDRESS__c (Locations) ===
# Only Address__c where:
# 1. Address__c does NOT have BBF_New_Id__c populated (not yet migrated)
# 2. Optionally: Only addresses used by active Orders

print("\n" + "=" * 80)
print("QUERYING ES ADDRESS__c (Locations)")
print("=" * 80)

if FILTER_ACTIVE_ORDERS_ONLY:
    # First get Address IDs used by active Orders
    print("\n📌 Getting Address IDs from active Orders...")
    order_query = """
        SELECT Address_A__c, Address_Z__c
        FROM Order
        WHERE Status = 'Activated'
          AND (Address_A__c != null OR Address_Z__c != null)
    """
    order_result = es_sf.query_all(order_query)

    # Collect unique address IDs
    address_ids = set()
    for order in order_result["records"]:
        if order.get("Address_A__c"):
            address_ids.add(order["Address_A__c"])
        if order.get("Address_Z__c"):
            address_ids.add(order["Address_Z__c"])

    print(f"   Found {len(address_ids)} unique addresses from active Orders")

    if len(address_ids) == 0:
        print("⚠️  No addresses found from active Orders")
        es_addresses = []
    else:
        # Query those specific addresses
        # SOQL IN clause has a limit, so we chunk if needed
        address_ids_list = list(address_ids)
        es_addresses = []
        chunk_size = 200

        for i in range(0, len(address_ids_list), chunk_size):
            chunk = address_ids_list[i : i + chunk_size]
            ids_str = "','".join(chunk)

            query = f"""
                SELECT Id, Name, 
                       Address__c, City__c, State__c, County__c, Zip__c,
                       Complete_Address__c, Clean_Street__c,
                       Geocode_Lat_Long__c, Geocode_Lat_Long__Latitude__s, Geocode_Lat_Long__Longitude__s,
                       CLLI__c, Building_Status__c, Building_Type__c,
                       On_Net__c, NNI__c, Headend__c,
                       Address_Type__c, Address_Status__c,
                       Country__c, County_FIPS__c,
                       RecordTypeId, OwnerId
                FROM Address__c
                WHERE Id IN ('{ids_str}')
                  AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')
            """

            if TEST_MODE and len(es_addresses) >= TEST_LIMIT:
                break

            result = es_sf.query_all(query)
            es_addresses.extend(result["records"])

            if TEST_MODE and len(es_addresses) >= TEST_LIMIT:
                es_addresses = es_addresses[:TEST_LIMIT]
                break
else:
    # Query all addresses not yet migrated
    query = """
        SELECT Id, Name, 
               Address__c, City__c, State__c, County__c, Zip__c,
               Complete_Address__c, Clean_Street__c,
               Geocode_Lat_Long__c, Geocode_Lat_Long__Latitude__s, Geocode_Lat_Long__Longitude__s,
               CLLI__c, Building_Status__c, Building_Type__c,
               On_Net__c, NNI__c, Headend__c,
               Address_Type__c, Address_Status__c,
               Country__c, County_FIPS__c,
               RecordTypeId, OwnerId
        FROM Address__c
        WHERE (BBF_New_Id__c = null OR BBF_New_Id__c = '')
    """

    # Add limit for test mode
    if TEST_MODE:
        query += f" LIMIT {TEST_LIMIT}"

    print(f"Query: {query[:500]}...")
    print("\nExecuting query...")

    result = es_sf.query_all(query)
    es_addresses = result["records"]

print(f"\n✅ Found {len(es_addresses)} Address__c records to migrate")

if len(es_addresses) > 0:
    sample = es_addresses[0]
    print(f"\nSample Address__c:")
    print(f"  ID:       {sample['Id']}")
    print(f"  Name:     {sample.get('Name', 'N/A')}")
    print(f"  Address:  {sample.get('Address__c', 'N/A')}")
    print(f"  City:     {sample.get('City__c', 'N/A')}")
    print(f"  State:    {sample.get('State__c', 'N/A')}")
    print(f"  Zip:      {sample.get('Zip__c', 'N/A')}")
elif TEST_MODE:
    print("\n⚠️  No unmigrated Address__c found in test set")
else:
    print(
        "\n✅ All Address__c records have been migrated (or no eligible records found)!"
    )


QUERYING ES ADDRESS__c (Locations)
Query: 
        SELECT Id, Name, 
               Address__c, City__c, State__c, County__c, Zip__c,
               Complete_Address__c, Clean_Street__c,
               Geocode_Lat_Long__c, Geocode_Lat_Long__Latitude__s, Geocode_Lat_Long__Longitude__s,
               CLLI__c, Building_Status__c, Building_Type__c,
               On_Net__c, NNI__c, Headend__c,
               Address_Type__c, Address_Status__c,
               Country__c, County_FIPS__c,
               RecordTypeId, OwnerId
        FROM Addr...

Executing query...

✅ Found 10 Address__c records to migrate

Sample Address__c:
  ID:       aD80B0000004CMvSAM
  Name:     ADR-0000400
  Address:  150 Mason Rd W
  City:     Milan
  State:    OH
  Zip:      44846-9799


In [18]:
# === TRANSFORM FOR BBF LOCATION__c ===

print("\n" + "=" * 80)
print("TRANSFORMING ADDRESS__c FOR BBF LOCATION__c")
print("=" * 80)


# State Translation: ES uses picklist, BBF uses string
# Usually direct mapping works, but we can add translations if needed
def translate_state(es_value):
    """Translate ES State__c to BBF format if needed."""
    if es_value is None or es_value == "":
        return None
    # Direct pass-through for now - add mappings if BBF has specific format requirements
    return es_value


bbf_locations = []
skipped_records = []

for es_addr in es_addresses:
    # Build BBF Location__c record
    # BBF Name_Is_Set_Manually__c is REQUIRED - default to False

    # Handle geolocation - BBF Loc__c expects latitude/longitude
    # ES stores as Geocode_Lat_Long__c with sub-fields
    bbf_geolocation = None
    es_lat = es_addr.get("Geocode_Lat_Long__Latitude__s")
    es_lng = es_addr.get("Geocode_Lat_Long__Longitude__s")
    if es_lat is not None and es_lng is not None:
        bbf_geolocation = {"latitude": es_lat, "longitude": es_lng}

    # Combine street address components if needed
    street = es_addr.get("Address__c") or es_addr.get("Clean_Street__c") or ""
    street = street.strip()[:255] if street else None  # Truncate to BBF field length

    # Full address - use Complete_Address__c or build from components
    full_address = es_addr.get("Complete_Address__c")
    if not full_address:
        # Build from components
        parts = [
            street,
            es_addr.get("City__c"),
            es_addr.get("State__c"),
            es_addr.get("Zip__c"),
        ]
        full_address = ", ".join([p for p in parts if p])
    full_address = (
        full_address[:255] if full_address else None
    )  # Truncate to field length

    bbf_location = {
        # Name field
        "Name": es_addr.get("Name", "Unknown Address")[:80],  # Max 80 chars
        # 🔴 REQUIRED: Must be set
        "Name_Is_Set_Manually__c": False,
        # Core address fields (exact matches)
        "City__c": es_addr.get("City__c"),
        "State__c": translate_state(es_addr.get("State__c")),
        "County__c": es_addr.get("County__c"),
        # Address fields (different API names)
        "PostalCode__c": es_addr.get("Zip__c"),
        "Street__c": street,
        "Full_Address__c": full_address,
        # Geolocation
        # "Loc__c": bbf_geolocation,
        # CLLI Code
        "CLLICode__c": es_addr.get("CLLI__c"),
        # Business Unit (BBF picklist)
        "businessUnit__c": DEFAULT_BUS_UNIT,
        # 👤 Set owner
        "OwnerId": OWNER_ID,
        # 🔗 Store ES Address ID for tracking
        "ES_Legacy_ID__c": es_addr["Id"],
    }

    # Remove None values to avoid API issues
    bbf_location = {k: v for k, v in bbf_location.items() if v is not None}

    # Ensure required field is present
    bbf_location["Name_Is_Set_Manually__c"] = False

    bbf_locations.append(bbf_location)

print(f"✅ Transformed {len(bbf_locations)} Locations")
if len(bbf_locations) > 0:
    sample_loc = bbf_locations[0]
    print(f"   - Mapped {len(sample_loc)} fields per Location")
    print(f"   - Set OwnerId to {OWNER_ID}")
    print(f"   - Set businessUnit__c to {DEFAULT_BUS_UNIT}")
    print(f"   - Set ES_Legacy_ID__c for tracking")
    print(f"\nSample transformed Location:")
    for k, v in list(sample_loc.items())[:8]:
        print(f"   {k}: {v}")

if len(skipped_records) > 0:
    print(f"\n⚠️  Skipped {len(skipped_records)} records")
    for skip in skipped_records[:5]:
        print(f"   - {skip['name']}: {skip['reason']}")


TRANSFORMING ADDRESS__c FOR BBF LOCATION__c
✅ Transformed 10 Locations
   - Mapped 11 fields per Location
   - Set OwnerId to 005Ea00000ZOGFZIA5
   - Set businessUnit__c to EVS
   - Set ES_Legacy_ID__c for tracking

Sample transformed Location:
   Name: ADR-0000400
   Name_Is_Set_Manually__c: False
   City__c: Milan
   State__c: OH
   County__c: Erie
   PostalCode__c: 44846-9799
   Street__c: 150 Mason Rd W
   Full_Address__c: 150 Mason Rd W, Milan, OH, 44846-9799


In [19]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING LOCATIONS TO BBF")
print("=" * 80)

if len(bbf_locations) == 0:
    print("⚠️  No Locations to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_locations)} Locations using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Location__c.insert(bbf_locations)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_locations[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_locations[i].get("Name", "N/A"),
                        "city": bbf_locations[i].get("City__c", "N/A"),
                        "state": bbf_locations[i].get("State__c", "N/A"),
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_locations[i]["ES_Legacy_ID__c"],
                        "name": bbf_locations[i].get("Name", "N/A"),
                        "errors": result["errors"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Locations")
        print(f"❌ Failed to insert: {len(failed_inserts)} Locations")

        if len(failed_inserts) > 0:
            print(f"\nFailed Locations (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES Address ID:   {sample['es_id']}")
            print(f"  BBF Location ID: {sample['bbf_id']}")
            print(f"  Name:            {sample['name']}")
            print(f"  City, State:     {sample['city']}, {sample['state']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING LOCATIONS TO BBF
Inserting 10 Locations using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 10 Locations
❌ Failed to insert: 0 Locations

Sample successful insert:
  ES Address ID:   aD80B0000004CMvSAM
  BBF Location ID: a1PEa00006xGCmXMAW
  Name:            ADR-0000400
  City, State:     Milan, OH


In [20]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Locations to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} Address__c records in ES...")
    print("   - Setting BBF_New_Id__c = BBF Location ID")

    try:
        es_update_results = es_sf.bulk.Address__c.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"\n✅ Successfully updated: {success_count} Address__c records in ES")
        print(f"❌ Failed to update: {error_count} Address__c records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []


UPDATING ES WITH BBF IDS
Updating 10 Address__c records in ES...
   - Setting BBF_New_Id__c = BBF Location ID

✅ Successfully updated: 10 Address__c records in ES
❌ Failed to update: 0 Address__c records in ES


In [21]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Status colors
status_colors = {
    "Success": "C6EFCE",  # Green
    "Failed": "FFC7CE",  # Red
    "Skipped": "FFEB9C",  # Yellow
}

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES Address ID",
    "BBF Location ID",
    "Name",
    "City",
    "State",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "Name": item["name"],
            "City": item["city"],
            "State": item["state"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "Name": item["name"],
            "City": "",
            "State": "",
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_records:
    all_results.append(
        {
            "ES_ID": item.get("es_id", ""),
            "BBF_ID": "",
            "Name": item.get("name", ""),
            "City": "",
            "State": "",
            "Status": "Skipped",
            "Error": item.get("reason", ""),
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_ID"],
            r["BBF_ID"],
            r["Name"],
            r["City"],
            r["State"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Location Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Filter Active Orders Only:", str(FILTER_ACTIVE_ORDERS_ONLY)])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append(["Business Unit:", DEFAULT_BUS_UNIT])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A9"].font = Font(bold=True)
ws2["B9"].font = Font(bold=True)
ws2.append(["Total Locations Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped Records", len(skipped_records)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Address ID", "BBF Location ID", "Name", "City", "State"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append(
        [item["es_id"], item["bbf_id"], item["name"], item["city"], item["state"]]
    )

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES Address ID", "Name", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append(
        [
            item["es_id"],
            item["name"],
            str(item["errors"]),
        ]
    )

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} Locations, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts)} failures)")


CREATING EXCEL OUTPUT

✅ Excel output saved to: es_bbf_location_migration_20251212_145550.xlsx
   📊 Sheet 1: Migration Results (10 Locations, color-coded)
   📈 Sheet 2: Summary (metrics and stats)
   🔗 Sheet 3: ID Mapping (10 successful mappings)
   ⚠️  Sheet 4: Failed Inserts (0 failures)


In [22]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Address__c queried: {len(es_addresses)}")
print(f"BBF Location__c inserted: {len(successful_inserts)}")
print(f"Failed inserts: {len(failed_inserts)}")
print(f"Skipped records: {len(skipped_records)}")
print(f"\nOutput file: {output_file}")

if TEST_MODE:
    print("\n⚠️  TEST MODE was enabled - only a subset of records were processed")
    print("   Set TEST_MODE = False to run full migration")

print("\n" + "=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("1. Review the Excel output for any failures")
print("2. Use the ID Mapping sheet for subsequent migrations (Service__c, etc.)")
print("3. If needed, run Node__c migration next (for Service__c.A_Node__c/Z_Node__c)")
print("4. Then proceed with Service__c migration using Location ID mappings")


MIGRATION COMPLETE
ES Address__c queried: 10
BBF Location__c inserted: 10
Failed inserts: 0
Skipped records: 0

Output file: es_bbf_location_migration_20251212_145550.xlsx

⚠️  TEST MODE was enabled - only a subset of records were processed
   Set TEST_MODE = False to run full migration

NEXT STEPS
1. Review the Excel output for any failures
2. Use the ID Mapping sheet for subsequent migrations (Service__c, etc.)
3. If needed, run Node__c migration next (for Service__c.A_Node__c/Z_Node__c)
4. Then proceed with Service__c migration using Location ID mappings


---
## Next Steps: Service Migration Prerequisites

After Location migration is complete, the following are needed for Service__c migration:

| Prerequisite | Status | Notes |
|--------------|--------|-------|
| Account | ✅ Complete | Account migration done |
| BAN__c | ✅ Complete | BAN migration done |
| Contact | ✅ Complete | Contact migration done |
| Location__c | 🔄 This notebook | Use ID Mapping sheet |
| Node__c | ❓ TBD | May not be needed - BBF Node is different concept |

### ID Mapping Files Needed for Service__c
- `es_bbf_account_migration_*.xlsx` → Account ID mapping
- `es_bbf_ban_migration_*.xlsx` → BAN ID mapping  
- `es_bbf_location_migration_*.xlsx` → Location ID mapping (this file)

In [23]:
# Install required packages (run if needed)
# !pip install simple-salesforce pandas openpyxl